In [2]:
from urllib.request import urlopen
import time 
from pyspark import SparkConf, SparkContext
from pyspark.sql.functions import *
from operator import add
from bs4 import BeautifulSoup
from urllib.error import HTTPError
import requests
import sys
import datetime

Url_Item method is used to opening sub page present in each main page. The following operations are perfomed in the method:

1)Open each sub page using BeautifulSoup

2)Fetch walmart model No and price

3)If any information scrapping fails for walmart, retry scraping.

4)If walmart model no is found, proceed for Amazon search with same model no

5)Scrap through amazon page and Check if the item exists in amazon, if yes scrap the price of the item. If no, attach message for the final list.


In [ ]:
def url_item(each,item_details,attempt):
    #opening of sub page 
    price_compare=""
    amz_search="https://www.amazon.com/s/ref=nb_sb_noss?url=search-alias%3Daps&field-keywords="
    try:
        subpage = urlopen(each)
        subpage_content = BeautifulSoup(subpage,"lxml")
        subpage_modelno=subpage_content.body.find("div",{"itemprop":"Model"})
        #capturing model no for walmart
        wal_model_no=subpage_modelno.text
        subpage_price=subpage_content.body.find("span",{"class":"price-group"})
        #capturing price of model in walmart
        wal_price= subpage_price.get("aria-label")
    except AttributeError as e:
        while attempt > 0:
            print("\nError(Attempt no: "+str(attempt)+" for url \'"+ str(each)+"\' is : Element Tag not found")
            attempt -=1
            #retry scrapping the sub page
            item_details = url_item(each,item_details,attempt)
    except:
        while attempt > 0:
            print("\nError(Attempt no: "+str(attempt)+" for url \'"+ str(each)+"\' is : "+str(sys.exc_info()[0]))
            attempt -=1
            #retry scrapping the sub page
            item_details = url_item(each,item_details,attempt)
    if wal_model_no is not None:# is walmart model is found, goahead and scrap amazon else skip it
        try:
             #goto amazon and scrap the page based on model no(fetched from walmart)
            amz_search = amz_search+wal_model_no
            headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.90 Safari/537.36'}
            amz_page = requests.get(amz_search,headers=headers)
            amz_cont = BeautifulSoup(amz_page.content,"lxml")
            amz_item = amz_cont.body.find("a",{"class":"a-link-normal a-text-normal"})
            amz_item_url = amz_item.get("href")
            #check if itme exist in amazon or not
            if "https://www.amazon.com" not in amz_item_url:
                price_compare = wal_price+"~Product Not Found"
                #print("Not found url")
            else:
                item_page = requests.get(amz_item_url,headers=headers)
                item_cont = BeautifulSoup(item_page.content,"lxml")
                amz_price = (item_cont.body.find("span",{"class":"a-size-medium a-color-price"})) #or (item_cont.body.find("div","class" : "a-section a-spacing-small a-spacing-top-small")) 
                price_compare = str(wal_price) + "~" + str(amz_price.text.strip())
            #append the model no, price into the final list    
            item_details.append(str(wal_model_no)+"~"+ str(price_compare))  
        except HTTPError as e:
            print("\nAmazon product not found for url \'"+ str(amz_search)+"\' is : "+str(e))
        except:
            print("\nAmazon product not found for url \'"+ str(amz_search)+"\' is : "+str(sys.exc_info()[0]))
      
    return(item_details) 

In [ ]:
def process(url):
    call_link=[] #individual page content
    item_details=[]
    try:
        #opening of page url under the main page
        page = urlopen(url)
        url_content = BeautifulSoup(page,"lxml")
    
##page content
        all_link = url_content.body.find_all("a",class_= desc)
        for link in all_link:
            call_link.append(subpage_url+link.get("href"))
        #opening sub pages
        for each in call_link:
            attempt = 2
            if each is None:
                continue
            try:
               item_details = url_item(each,item_details,attempt) 
            except:
                print("\nError for url \'"+ str(each)+"\' is : "+str(sys.exc_info()[0]))
    except:
        print("error"+str(sys.exc_info()[0]))
    return item_details


In [ ]:
#list of items
category = ['microwave','juicer','TV','Cellphones','laptops']   
for each in category:
    url = "https://www.walmart.com/search/?query="+each+"&cat_id=0"
    desc=[]
    desc.append('product-title-link')
    desc.append('line-clamp')
    
    desc.append('line-clamp-2')
    subpage_url="https://www.walmart.com"
    wal_url = "https://www.walmart.com/search/?query="+each+"&page="
    wal_end_url = "&cat_id=0&grid=true&ps=40"
    call_link=[] #individual page content
    new_page=[]#page url
    list_item = {}
    page = urlopen(url)
    soup = BeautifulSoup(page,"lxml")
    #retrieving page nos
    paginator = soup.body.find(class_="paginator-list")
    page_no = paginator.find(class_= "active")
    page_no = page_no.get("aria-label").split()[3]
    print("total no of pages "+ str(page_no))
    #forming page url
    for i in range(1,int(page_no)+1):
        new_page.append(wal_url+str(i)+wal_end_url)
    print("length is" + str(len(new_page)))
    #creation of rdd
    start = time.time()
    spark = SparkContext.getOrCreate()
    spark.setLogLevel("WARN")
    rdd1 = spark.parallelize(new_page)
    rdd2 = rdd1.repartition(4)
    list_item = rdd2.flatMap(lambda url : process(url))
    #saving outfile file for each catgeory with current time stamp
    list_item.saveAsTextFile('/price_webscrapping_'+str(datetime.datetime.now())[0:16].replace(':','_').replace(' ','_')+'/')
    #prints out the time taken to process each category
    print("Total time is " + str(time.time() - start) + " in ms" )
    
    #end of rdd creation
